In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

### 범주형 > 수치형 : 라벨인코딩

In [55]:
df2 = pd.DataFrame([
                    ['green', 'M', 10.1, 'class1'],
                    ['red', 'L', 13.5, 'class2'],
                    ['blue', 'XL', 15.3, 'class1'],
                ])
df2.columns = ['color', 'size', 'price', 'classlabel']
df2

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [56]:
# np.unique(df2['classlabel'], return_counts = True)
# df2['classlabel'].unique()

# class_mapping = { 'class1' : 0, 'class2' : 1, 'class3' : 2} # 순위형 범주 데이터
class_mapping = {
                    label : idx for idx, label
                    in enumerate(np.unique(df2['classlabel']))
                }
class_mapping

df2['classlabel2'] = df2['classlabel'].map(class_mapping)
df2

,color,size,price,classlabel,classlabel2
0,green,M,10.1,class1,0
1,red,L,13.5,class2,1
2,blue,XL,15.3,class1,0


In [57]:
# v, k 순서 주의
# 수치형으로 바꿨던 범주형의 값을 다시 원상태(범주형)으로 쓰기
inv_class_mapping = { v : k for k, v in class_mapping.items() }
print(inv_class_mapping)
df2['classlabel2'] = df2['classlabel2'].map(inv_class_mapping)
df2

{0: 'class1', 1: 'class2'}


,color,size,price,classlabel,classlabel2
0,green,M,10.1,class1,class1
1,red,L,13.5,class2,class2
2,blue,XL,15.3,class1,class1


범주형 데이터 처리

In [58]:
from sklearn.preprocessing import LabelEncoder  # 범주형 -> 순위형 변경 도구
# 사이킷런의 라벨인코더가 범주형 > 수치형으로 바꿔주는 도구

class_le = LabelEncoder()
print( df2['classlabel'].values )
# y = class_le.fit_transform(df2['classlabel'].values)  # fit + transform
class_le.fit(df2['classlabel'].values)      # fit만
y = class_le.transform(df2['classlabel'].values)    # transform만
y

['class1' 'class2' 'class1']


array([0, 1, 0])

In [59]:
df2['classlabel'] = class_le.fit_transform(df2['classlabel'].values)
df2

,color,size,price,classlabel,classlabel2
0,green,M,10.1,0,class1
1,red,L,13.5,1,class2
2,blue,XL,15.3,0,class1


In [60]:
# class_le.inverse_transform(df2['classlabel'].values)

df2['classlabel'] = class_le.inverse_transform(df2['classlabel'].values)
df2

,color,size,price,classlabel,classlabel2
0,green,M,10.1,class1,class1
1,red,L,13.5,class2,class2
2,blue,XL,15.3,class1,class1


### 범주형 > 수치형 : 원핫인코딩

In [61]:
from sklearn.preprocessing import OneHotEncoder

In [62]:
X = df2[['color', 'price']].values
X
# color는 범주형이지만, 순위형이 아니므로 labelEncoding하면 안됌

array([['green', 10.1],
       ['red', 13.5],
       ['blue', 15.3]], dtype=object)

In [63]:
color_le = LabelEncoder()   # LabelEncoder는 색상에 적절하게 인코딩 되지 않음을 확인 > 1, 2, 0 순위 값
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 10.1],
       [2, 13.5],
       [0, 15.3]], dtype=object)

In [64]:
ohe = OneHotEncoder()   # 3개이므로, 3컬럼을 생성한 뒤 0으로 채우고, 1, 2, 0 번째를 1로 바꿈 > 순위가 없음
encoded = ohe.fit_transform(X[:, [0]]).toarray()    # 모든 행의 0번째 컬럼만 Encoding
print(encoded)

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [65]:
# 판다스에 get_dummies로도 원핫 인코딩 가능함
pd.get_dummies(df2[['price', 'color']])
# price는 숫자라서 바꾸지 않고, color는 문자라서 이 것 만 T/F로 표시하는데, 이게 1/0인 원핫인코딩임

,price,color_blue,color_green,color_red
0,10.1,False,True,False
1,13.5,False,False,True
2,15.3,True,False,False


### 훈련/테이스 데이터 분할

In [101]:
url_df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',
                      header=None)
url_df_wine.columns = ['Class label', 'Alcool', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']
csv_df_wine = pd.read_csv('data-files/winequality-white.csv', sep=";")
url_df_wine.head(3), csv_df_wine.head(3)

(   Class label  Alcool  Malic acid   Ash  Alcalinity of ash  Magnesium  \
 0            1   14.23        1.71  2.43               15.6        127   
 1            1   13.20        1.78  2.14               11.2        100   
 2            1   13.16        2.36  2.67               18.6        101   
 
    Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins  \
 0           2.80        3.06                  0.28             2.29   
 1           2.65        2.76                  0.26             1.28   
 2           2.80        3.24                  0.30             2.81   
 
    Color intensity   Hue  OD280/OD315 of diluted wines  Proline  
 0             5.64  1.04                          3.92     1065  
 1             4.38  1.05                          3.40     1050  
 2             5.68  1.03                          3.17     1185  ,
    fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
 0            7.0              0.27         0.36            

In [102]:
from sklearn.model_selection import train_test_split

In [103]:
# csv파일로 작업 - quality
X, y = csv_df_wine.iloc[:, :-1].values, csv_df_wine.iloc[:, -1].values

# X_train, X_test, y_train, y_test = \
#             train_test_split(X, y, test_size=0.3, random_state=42)
# 이렇게하면 test에서 9는 빠짐
X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
            # stratify=y: y의 분포를 훈련/데이터 나눌 때 지켜라 
            # stratify는 범주형데이터에서만 유의미하다

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3428, 11), (1470, 11), (3428,), (1470,))

In [104]:
np.unique(y, return_counts=True)

(array([3, 4, 5, 6, 7, 8, 9]),
 array([  20,  163, 1457, 2198,  880,  175,    5]))

In [105]:
np.unique(y_train, return_counts=True)

(array([3, 4, 5, 6, 7, 8, 9]),
 array([  14,  114, 1020, 1538,  616,  122,    4]))

In [106]:
# 9번이 빠져버림 - 수정함
np.unique(y_test, return_counts=True)

(array([3, 4, 5, 6, 7, 8, 9]), array([  6,  49, 437, 660, 264,  53,   1]))

In [107]:
# url로 받은 데이터로 작업
X, y = url_df_wine.iloc[:, 1:].values, url_df_wine.iloc[:, 0].values

X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
            # stratify는 범주형데이터에서만 유의미하다

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((124, 13), (54, 13), (124,), (54,))

In [108]:
np.unique(y, return_counts=True)

(array([1, 2, 3]), array([59, 71, 48]))

In [109]:
np.unique(y_train, return_counts=True)

(array([1, 2, 3]), array([41, 50, 33]))

In [110]:
np.unique(y_test, return_counts=True)

(array([1, 2, 3]), array([18, 21, 15]))